In [1]:
%matplotlib inline
import copy
import time
import tqdm
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms,datasets, models



1.9.0+cu102


## Load Data:

In [2]:
apply_transform = transforms.Compose([transforms.Resize(224),transforms.ToTensor()])
BatchSize = 32

trainset = datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=apply_transform)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=apply_transform)
testLoader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [3]:
# Size of train and test datasets
print('No. of samples in train set: '+str(len(trainLoader.dataset)))
print('No. of samples in test set: '+str(len(testLoader.dataset)))

No. of samples in train set: 50000
No. of samples in test set: 10000


## Define network architecture

In [4]:
net1 = models.resnet18() # Training from scratch
net2 = models.resnet18(pretrained=True) # End-to-end fine-tuning
net3 = models.resnet18(pretrained=True) # Training only the last layer
print(net1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
# Counting number of trainable parameters
totalParams = 0
for name,params in net1.named_parameters():
    print(name,'-->',params.size())
    totalParams += np.sum(np.prod(params.size()))
print('Total number of parameters: '+str(totalParams))

conv1.weight --> torch.Size([64, 3, 7, 7])
bn1.weight --> torch.Size([64])
bn1.bias --> torch.Size([64])
layer1.0.conv1.weight --> torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight --> torch.Size([64])
layer1.0.bn1.bias --> torch.Size([64])
layer1.0.conv2.weight --> torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight --> torch.Size([64])
layer1.0.bn2.bias --> torch.Size([64])
layer1.1.conv1.weight --> torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight --> torch.Size([64])
layer1.1.bn1.bias --> torch.Size([64])
layer1.1.conv2.weight --> torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight --> torch.Size([64])
layer1.1.bn2.bias --> torch.Size([64])
layer2.0.conv1.weight --> torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight --> torch.Size([128])
layer2.0.bn1.bias --> torch.Size([128])
layer2.0.conv2.weight --> torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight --> torch.Size([128])
layer2.0.bn2.bias --> torch.Size([128])
layer2.0.downsample.0.weight --> torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight 

In [6]:
# Modifying the last fully-connected layer for 10 classes
net1.fc = nn.Linear(512,10)
net2.fc = nn.Linear(512,10)
net3.fc = nn.Linear(512,10)

In [7]:
# Copying initial weights for visualization
# Model 1
init_weightConv1_1 = copy.deepcopy(net1.conv1.weight.data) # 1st conv layer
init_weightConv2_1 = copy.deepcopy(net1.layer1[0].conv1.weight.data) # 2nd conv layer
# Model 2
init_weightConv1_2 = copy.deepcopy(net2.conv1.weight.data) # 1st conv layer
init_weightConv2_2 = copy.deepcopy(net2.layer1[0].conv1.weight.data) # 2nd conv layer
# Model 3
init_weightConv1_3 = copy.deepcopy(net3.conv1.weight.data) # 1st conv layer
init_weightConv2_3 = copy.deepcopy(net3.layer1[0].conv1.weight.data) # 2nd conv layer

In [8]:
# Check availability of GPU

use_gpu = torch.cuda.is_available()
# use_gpu = False # Uncomment in case of GPU memory error
if use_gpu:
    print('GPU is available!')
    device = "cuda"
else:
    print('GPU is not available!')
    device = "cpu"
    
net1 = net1.to(device)
net2 = net2.to(device)
net3 = net3.to(device)

GPU is available!


## Define loss function and optimizer

In [9]:
criterion = nn.NLLLoss() # Negative Log-likelihood
optimizer1 = optim.Adam(net1.parameters(), lr=1e-4) # Adam; passing all params
optimizer2 = optim.Adam(net2.parameters(), lr=1e-4) # Adam; passing all params
optimizer3 = optim.Adam(net3.fc.parameters(), lr=1e-4) # Adam; passing params of only the last fc layer

## Train the network

In [ ]:
iterations = 10

trainLoss1 = []
trainTotalLoss1 = []
trainAcc1 = []
testLoss1 = []
testAcc1 = []

trainLoss2 = []
trainTotalLoss2 = []
trainAcc2 = []
testLoss2 = []
testAcc2 = []

trainLoss3 = []
trainTotalLoss3 = []
trainAcc3 = []
testLoss3 = []
testAcc3 = []


start = time.time()
for epoch in range(iterations):
    epochStart = time.time()
    # Model 1
    runningLoss1 = 0.0   
    avgTotalLoss1 = 0.0
    running_correct1 = 0
    # Model 2
    runningLoss2 = 0.0   
    avgTotalLoss2 = 0.0
    running_correct2 = 0
    # Model 3
    runningLoss3 = 0.0   
    avgTotalLoss3 = 0.0
    running_correct3 = 0
    
    net1.train() # For training   
    net2.train() # For training   
    net3.train() # For training   
    for data in tqdm.tqdm_notebook(trainLoader):
        inputs,labels = data
        
        inputs, labels = inputs.to(device), labels.to(device)  
        # Model 1
        outputs1 = net1(inputs)
        _, predicted1 = torch.max(outputs1.data, 1)
        running_correct1 += (predicted1 == labels.data).sum()
        # Model 2
        outputs2 = net2(inputs)
        _, predicted2 = torch.max(outputs2.data, 1)
        running_correct2 += (predicted2 == labels.data).sum()
        # Model 3
        outputs3 = net3(inputs)
        _, predicted3 = torch.max(outputs3.data, 1)
        running_correct3 += (predicted3 == labels.data).sum()
       
        # Initialize gradients to zero
        optimizer1.zero_grad()               
        optimizer2.zero_grad()               
        optimizer3.zero_grad()               
        
        # Compute loss/error
        loss1 = criterion(F.log_softmax(outputs1,dim=1), labels)
        loss2 = criterion(F.log_softmax(outputs2,dim=1), labels)
        loss3 = criterion(F.log_softmax(outputs3,dim=1), labels)
        # Backpropagate loss and compute gradients
        loss1.backward()
        loss2.backward()
        loss3.backward()
        # Update the network parameters
        optimizer1.step()
        optimizer2.step()
        optimizer3.step()
        # Accumulate loss per batch
        runningLoss1 += loss1.item()         
        runningLoss2 += loss2.item()         
        runningLoss3 += loss3.item() 
    
    avgTrainAcc1 = 100*float(running_correct1)/50000.0
    avgTrainLoss1 = runningLoss1/(50000.0/BatchSize)    
    trainAcc1.append(avgTrainAcc1)
    trainLoss1.append(avgTrainLoss1)    
    #-------------------------------
    avgTrainAcc2 = 100*float(running_correct2)/50000.0
    avgTrainLoss2 = runningLoss2/(50000.0/BatchSize)    
    trainAcc2.append(avgTrainAcc2)
    trainLoss2.append(avgTrainLoss2)    
    #-------------------------------
    avgTrainAcc3 = 100*float(running_correct3)/50000.0
    avgTrainLoss3 = runningLoss3/(50000.0/BatchSize)    
    trainAcc3.append(avgTrainAcc3)
    trainLoss3.append(avgTrainLoss3)
    
    # Evaluating performance on test set for each epoch
    net1.eval() # For testing [Affects batch-norm and dropout layers (if any)]
    net2.eval()
    net3.eval()
    running_correct1 = 0  
    running_correct2 = 0  
    running_correct3 = 0  
    with torch.no_grad():
        for data in testLoader:
            inputs,labels = data            
            inputs, labels = inputs.to(device), labels.to(device)
            # Model 1
            outputs1 = net1(inputs)
            _, predicted1 = torch.max(outputs1.data, 1)
            running_correct1 += (predicted1 == labels.data).sum()
            # Model 2
            outputs2 = net2(inputs)
            _, predicted2 = torch.max(outputs2.data, 1)
            running_correct2 += (predicted2 == labels.data).sum()
            # Model 3
            outputs3 = net3(inputs)
            _, predicted3 = torch.max(outputs3.data, 1)
            running_correct3 += (predicted3 == labels.data).sum()

            loss1 = criterion(F.log_softmax(outputs1,dim=1), labels)
            loss2 = criterion(F.log_softmax(outputs2,dim=1), labels)
            loss3 = criterion(F.log_softmax(outputs3,dim=1), labels)

            runningLoss1 += loss1.item()         
            runningLoss2 += loss2.item()         
            runningLoss3 += loss3.item() 
            
    avgTestLoss1 = runningLoss1/(10000.0/BatchSize)
    avgTestAcc1 = 100*float(running_correct1)/10000.0
    testLoss1.append(avgTestLoss1)
    testAcc1.append(avgTestAcc1)
    #---------------------------
    avgTestLoss2 = runningLoss2/(10000.0/BatchSize)
    avgTestAcc2 = 100*float(running_correct2)/10000.0
    testLoss2.append(avgTestLoss2)
    testAcc2.append(avgTestAcc2)
    #---------------------------
    avgTestLoss3 = runningLoss3/(10000.0/BatchSize)
    avgTestAcc3 = 100*float(running_correct3)/10000.0
    testLoss3.append(avgTestLoss3)
    testAcc3.append(avgTestAcc3)
        
    
    # Plotting training loss vs Epochs
    fig1 = plt.figure(1)        
    plt.plot(range(epoch+1),trainLoss1,'r-',label='train')  
    plt.plot(range(epoch+1),testLoss1,'g-',label='test') 
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss of Model 1')   
    # Plotting testing accuracy vs Epochs
    fig2 = plt.figure(2)        
    plt.plot(range(epoch+1),trainAcc1,'r-',label='train')    
    plt.plot(range(epoch+1),testAcc1,'g-',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy of Model 1')   
        
    # Plotting training loss vs Epochs
    fig3 = plt.figure(3)        
    plt.plot(range(epoch+1),trainLoss2,'r-',label='train')  
    plt.plot(range(epoch+1),testLoss2,'g-',label='test') 
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss of Model 2')   
    # Plotting testing accuracy vs Epochs
    fig4 = plt.figure(4)        
    plt.plot(range(epoch+1),trainAcc2,'r-',label='train')    
    plt.plot(range(epoch+1),testAcc2,'g-',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy of Model 2')
        
    # Plotting training loss vs Epochs
    fig5 = plt.figure(5)        
    plt.plot(range(epoch+1),trainLoss3,'r-',label='train')  
    plt.plot(range(epoch+1),testLoss3,'g-',label='test') 
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss of Model 1')   
    # Plotting testing accuracy vs Epochs
    fig6 = plt.figure(6)        
    plt.plot(range(epoch+1),trainAcc3,'r-',label='train')    
    plt.plot(range(epoch+1),testAcc3,'g-',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy of Model 1')
        
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f} Model 1  ;  Training Loss: {:.6f} ; Testing Acc: {:.3f}'\
          .format(epoch + 1,iterations,avgTrainLoss1,avgTestAcc1))
    print('Iteration: {:.0f} /{:.0f} Model 2  ;  Training Loss: {:.6f} ; Testing Acc: {:.3f} '\
          .format(epoch + 1,iterations,avgTrainLoss2,avgTestAcc2))
    print('Iteration: {:.0f} /{:.0f} Model 3  ;  Training Loss: {:.6f} ; Testing Acc: {:.3f} '\
      .format(epoch + 1,iterations,avgTrainLoss3,avgTestAcc3))
    print('Time consumed: {:.0f}m {:.0f}s'.format(epochEnd//60,epochEnd%60))
end = time.time()-start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1563 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Iteration: 1 /10 Model 1  ;  Training Loss: 1.242137 ; Testing Acc: 60.820
Iteration: 1 /10 Model 2  ;  Training Loss: 0.334131 ; Testing Acc: 92.890 
Iteration: 1 /10 Model 3  ;  Training Loss: 1.384695 ; Testing Acc: 73.700 
Time consumed: 19m 3s


  0%|          | 0/1563 [00:00<?, ?it/s]

...

  0%|          | 0/1563 [00:00<?, ?it/s]

Iteration: 4 /10 Model 1  ;  Training Loss: 0.437424 ; Testing Acc: 78.680
Iteration: 4 /10 Model 2  ;  Training Loss: 0.058039 ; Testing Acc: 94.040 
Iteration: 4 /10 Model 3  ;  Training Loss: 0.710847 ; Testing Acc: 78.560 
Time consumed: 18m 59s


  0%|          | 0/1563 [00:00<?, ?it/s]

Iteration: 5 /10 Model 1  ;  Training Loss: 0.310333 ; Testing Acc: 78.890
Iteration: 5 /10 Model 2  ;  Training Loss: 0.046162 ; Testing Acc: 93.250 
Iteration: 5 /10 Model 3  ;  Training Loss: 0.680050 ; Testing Acc: 79.240 
Time consumed: 18m 56s


  0%|          | 0/1563 [00:00<?, ?it/s]

In [ ]:
# Plotting training loss vs Epochs
fig7 = plt.figure(7)        
plt.plot(range(epoch+1),trainLoss1,'r-',label='model1')  
plt.plot(range(epoch+1),trainLoss2,'g-',label='model2') 
plt.plot(range(epoch+1),trainLoss3,'b-',label='model3') 
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Train Loss')  

fig8 = plt.figure(8)        
plt.plot(range(epoch+1),testLoss1,'r-',label='model1')  
plt.plot(range(epoch+1),testLoss2,'g-',label='model2') 
plt.plot(range(epoch+1),testLoss3,'b-',label='model3') 
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Test Loss') 

fig9 = plt.figure(9)        
plt.plot(range(epoch+1),testAcc1,'r-',label='model1')  
plt.plot(range(epoch+1),testAcc2,'g-',label='model2') 
plt.plot(range(epoch+1),testAcc3,'b-',label='model3') 
plt.legend(loc='upper left')
plt.xlabel('Epochs')
plt.ylabel('Test Accuracy') 

In [ ]:
# Copying trained weights for visualization
trained_weightConv1_1 = copy.deepcopy(net1.conv1.weight.data)
trained_weightConv2_1 = copy.deepcopy(net1.layer1[0].conv1.weight.data)

trained_weightConv1_2 = copy.deepcopy(net2.conv1.weight.data)
trained_weightConv2_2 = copy.deepcopy(net2.layer1[0].conv1.weight.data)

trained_weightConv1_3 = copy.deepcopy(net3.conv1.weight.data)
trained_weightConv2_3 = copy.deepcopy(net3.layer1[0].conv1.weight.data)
if use_gpu:
    trained_weightConv1_1 = trained_weightConv1_1.cpu()
    trained_weightConv2_1 = trained_weightConv2_1.cpu()
    
    trained_weightConv1_2 = trained_weightConv1_2.cpu()
    trained_weightConv2_2 = trained_weightConv2_2.cpu()
    
    trained_weightConv1_3 = trained_weightConv1_3.cpu()
    trained_weightConv2_3 = trained_weightConv2_3.cpu()

## Visualization of weights

In [ ]:
# functions to show an image
def imshow(img, strlabel):
    npimg = img.numpy()
    npimg = np.abs(npimg)
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 10
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    plt.figure()
    plt.title(strlabel)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# Visualizing weights of 1st convolutional layer of Model 1
imshow(torchvision.utils.make_grid(init_weightConv1_1,nrow=8,normalize=True),'Initial weights: conv1')
imshow(torchvision.utils.make_grid(trained_weightConv1_1,nrow=8,normalize=True),'Trained weights: conv1')
imshow(torchvision.utils.make_grid(init_weightConv1_1-trained_weightConv1_1,nrow=8,normalize=True),'Difference of weights: conv1')

In [ ]:
# Visualizing weights of 2nd convolutional layer of Model 1
imshow(torchvision.utils.make_grid(init_weightConv2_1[0].unsqueeze(1),nrow=8,normalize=True),'Initial weights: conv2')
imshow(torchvision.utils.make_grid(trained_weightConv2_1[0].unsqueeze(1),nrow=8,normalize=True),'Trained weights: conv2')
imshow(torchvision.utils.make_grid(init_weightConv2_1[0].unsqueeze(1)-trained_weightConv2_1[0].unsqueeze(1),nrow=8,normalize=True),'Difference of weights: conv2')

In [ ]:
# Visualizing weights of 1st convolutional layer of Model 2
imshow(torchvision.utils.make_grid(init_weightConv1_2,nrow=8,normalize=True),'Initial weights: conv1')
imshow(torchvision.utils.make_grid(trained_weightConv1_2,nrow=8,normalize=True),'Trained weights: conv1')
imshow(torchvision.utils.make_grid(init_weightConv1_2-trained_weightConv1_2,nrow=8,normalize=True),'Difference of weights: conv1')

In [ ]:
# Visualizing weights of 2nd convolutional layer of Model 2
imshow(torchvision.utils.make_grid(init_weightConv2_2[0].unsqueeze(1),nrow=8,normalize=True),'Initial weights: conv2')
imshow(torchvision.utils.make_grid(trained_weightConv2_2[0].unsqueeze(1),nrow=8,normalize=True),'Trained weights: conv2')
imshow(torchvision.utils.make_grid(init_weightConv2_2[0].unsqueeze(1)-trained_weightConv2_2[0].unsqueeze(1),nrow=8,normalize=True),'Difference of weights: conv2')

In [ ]:
# Visualizing weights of 1st convolutional layer of Model 3
imshow(torchvision.utils.make_grid(init_weightConv1_3,nrow=8,normalize=True),'Initial weights: conv1')
imshow(torchvision.utils.make_grid(trained_weightConv1_3,nrow=8,normalize=True),'Trained weights: conv1')
imshow(torchvision.utils.make_grid(init_weightConv1_3-trained_weightConv1_3,nrow=8,normalize=True),'Difference of weights: conv1')

In [ ]:
# Visualizing weights of 2nd convolutional layer of Model 3
imshow(torchvision.utils.make_grid(init_weightConv2_3[0].unsqueeze(1),nrow=8,normalize=True),'Initial weights: conv2')
imshow(torchvision.utils.make_grid(trained_weightConv2_3[0].unsqueeze(1),nrow=8,normalize=True),'Trained weights: conv2')
imshow(torchvision.utils.make_grid(init_weightConv2_3[0].unsqueeze(1)-trained_weightConv2_3[0].unsqueeze(1),nrow=8,normalize=True),'Difference of weights: conv2')